In [2]:
import time
import scipy.special
import numpy as np
import matplotlib.pyplot as plt
from pyscf import gto, scf, fci, dft

# set the basis
basis = '6-31G'

# define molecule parameters: H2
bond_length = 0.702
geometry = 'H 0 0 {}; H 0 0 {}'
spin = 0

# geometry = '''
# C          1.13569        2.22607        0.00008
# C          2.34660        1.48225       -0.00315
# O          1.11076        3.45322        0.00297
# N          0.00000        1.41087        0.00000
# N          3.65481        1.87305       -0.00480
# C          2.38646        0.12104       -0.00500
# N          1.24110       -0.63754       -0.00365
# N          3.66420       -0.35109       -0.00778
# C          0.00000        0.00000        0.00000
# O         -1.05949       -0.63408        0.00333
# C         -1.30072        2.05116        0.00361
# C          1.32846       -2.08635       -0.00430
# H         -1.84850        1.73732        0.89829
# H         -1.86170        1.72312       -0.87769
# H         -1.23294        3.14176       -0.00582
# H          1.87131       -2.41138       -0.89754
# H          0.34073       -2.55462       -0.00693
# H          1.86704       -2.41210        0.89126
# C          4.41443        0.73303       -0.00765
# C          4.15051        3.22594       -0.00368
# H          3.78631        3.73087       -0.90182
# H          5.24349        3.21375       -0.00572
# H          3.78964        3.72816        0.89731
# H          5.49723        0.74732       -0.00947
# '''
# spin = 0

# build the molecule and set the verbosity level
mol = gto.M(atom    = geometry.format(-0.5 * bond_length, 0.5 * bond_length),
            basis   = basis,
            spin    = spin,
            verbose = 0)

# run the HF calculations
start_time = time.time()
hf_calc = scf.UHF(mol)
res = hf_calc.scf()
elapsed_time = time.time() - start_time
print('{:32s}: {:.4f} Ha ({:.2f} seconds)'.format('HF energy', res, elapsed_time))

# run the FCI calculations
start_time = time.time()
hf_calc = scf.RHF(mol) # we cheat a bit here because the pyscf CI solver only accepts RHF
hf_calc.kernel()
ci_calc = fci.FCI(mol, hf_calc.mo_coeff)
res = ci_calc.kernel()
res = res[0] # extract the ground state energy
elapsed_time = time.time() - start_time
print('{:32s}: {:.4f} Ha ({:.2f} seconds)'.format('Full CI energy', res, elapsed_time))

#run DFT calculations
for functional in ['LDA', 'PBE', 'HSE06', 'BLYP', 'B3LYP']:
    start_time = time.time()
    dft_calc = dft.KS(mol)
    dft_calc.xc = functional
    res = dft_calc.kernel()
    elapsed_time = time.time() - start_time
    print('{:32s}: {:.4f} Ha ({:.2f} seconds)'.format('{} energy'.format(functional), res, elapsed_time))

HF energy                       : -1.1262 Ha (0.04 seconds)
Full CI energy                  : -1.1503 Ha (0.03 seconds)
LDA energy                      : -1.0355 Ha (0.06 seconds)
PBE energy                      : -1.1603 Ha (0.07 seconds)
HSE06 energy                    : -1.1629 Ha (0.21 seconds)
BLYP energy                     : -1.1637 Ha (0.07 seconds)
B3LYP energy                    : -1.1675 Ha (0.08 seconds)
